## Notebook to annotate DEGlist from 7control V 7armdrop exposed comparison for SICB 2024.  

### Files Needed:
1. `BLAST` output from the _Pycnopodia helianothoides_ genome gene list: https://raw.githubusercontent.com/grace-ac/project-pycno-sizeclass-2022/main/analyses/10-BLAST/summer2022-uniprot_blastx_sep.tab 
2. DEGlist from comparing 7 control stars to 7 arm drop exposed stars for SICB: https://raw.githubusercontent.com/grace-ac/project-pycno-sizeclass-2022/main/analyses/08-deseq2/DEGlist_control_v_armdrop_7x7.tab
3. Uniprot GO Annotation file: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted
4. GOslim file available here: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted


In [2]:
pwd

'/Users/graciecrandall/Documents/GitHub/project-pycno-sizeclass-2022/code'

In [3]:
#change working directory to analyses folder for this notebook
wd = "../analyses/14-annotate-deglist-blastGO/"

In [4]:
cd $wd

/Users/graciecrandall/Documents/GitHub/project-pycno-sizeclass-2022/analyses/14-annotate-deglist-blastGO


In [6]:
# curl `blast` ouptut from Phel genome gene list
!curl --insecure https://raw.githubusercontent.com/grace-ac/project-pycno-sizeclass-2022/main/analyses/10-BLAST/summer2022-uniprot_blastx_sep.tab \
    -o Phel.genelist.BLASTx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1435k  100 1435k    0     0  2817k      0 --:--:-- --:--:-- --:--:-- 2837k


In [7]:
# curl uniprot-sprot sorted
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted -o uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2968  100  2968    0     0  11832      0 --:--:-- --:--:-- --:--:-- 14914


In [8]:
# curl GO-GOslim sorted
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted -o GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2968  100  2968    0     0  64395      0 --:--:-- --:--:-- --:--:-- 82444


In [10]:
#check that curl-ed files are in directory
#accidentally curled in 2015.Phel.BLASTX... ignore
!ls

2015.Phel.BLASTx     Phel.genelist.BLASTx
GO-GOslim.sorted     uniprot-SP-GO.sorted


In [11]:
!sort -u -k1,1 --field-separator $'\t' Phel.genelist.BLASTx > blastout

In [12]:
!wc -l blastout

   13604 blastout


In [13]:
#set `blast` output file as variable
blastout="Phel.genelist.BLASTx"

In [14]:
!head -2 blastout

In [15]:
#convert pipes to tab
!tr '|' '\t' < blastout \
> _blast-sep.tab

In [16]:
!head -2 _blast-sep.tab

In [17]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [18]:
!head -2 _blast-sort.tab

A0A061ACU2	g2123.t1	0
A0A061ACU2	g2123.t2	0


In [19]:
!head -2 uniprot-SP-GO.sorted

<!DOCTYPE html>
<html>


For some reason the uniprot-SP-GO.sorted and GO-GOslim.sorted are .html files... but I have those same files in a github repo from my crab stuff so I'll just use those for now. 

In [20]:
!head -2 uniprot-SP-GO.sorted

A0A023GPI8	LECA_CANBL	reviewed	Lectin alpha chain (CboL) [Cleaved into: Lectin beta chain; Lectin gamma chain]		Canavalia boliviana	237			mannose binding [GO:0005537]; metal ion binding [GO:0046872]	mannose binding [GO:0005537]; metal ion binding [GO:0046872]	GO:0005537; GO:0046872
A0A023GPJ0	CDII_ENTCC	reviewed	Immunity protein CdiI	cdiI ECL_04450.1	Enterobacter cloacae subsp. cloacae (strain ATCC 13047 / DSM 30054 / NBRC 13535 / NCDC 279-56)	145					


In [21]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [22]:
!head -2 _blast-annot.tab

A0A0D2YG10	g13111.t1	GO:0016491; GO:0016746; GO:0031177
A0A0D3QS98	g5914.t1	GO:0005576; GO:0016055; GO:0030178; GO:1990697; GO:1990699


The following is a script modified by Sam White

In [23]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [24]:
!head _blast-GO-unfolded.tab

A0A0D2YG10	g13111.t1	GO:0016491
A0A0D2YG10	g13111.t1	GO:0016746
A0A0D2YG10	g13111.t1	GO:0031177
A0A0D3QS98	g5914.t1	GO:0005576
A0A0D3QS98	g5914.t1	GO:0016055
A0A0D3QS98	g5914.t1	GO:0030178
A0A0D3QS98	g5914.t1	GO:1990697
A0A0D3QS98	g5914.t1	GO:1990699
A0A0F7Z3J2	g4163.t1	GO:0005179
A0A0F7Z3J2	g4163.t1	GO:0005576


In [25]:
#gets rid of lines with no GOIDs
!sort -k3 _blast-GO-unfolded.tab | grep "GO:" > _blast-GO-unfolded.sorted

In [26]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [27]:
!head Blastquery-GOslim.tab

g9026.t1	GO:0000002	cell organization and biogenesis	P
g1868.t1	GO:0000002	cell organization and biogenesis	P
g9382.t1	GO:0000002	cell organization and biogenesis	P
g16208.t1	GO:0000002	cell organization and biogenesis	P
g4483.t1	GO:0000002	cell organization and biogenesis	P
g4483.t2	GO:0000002	cell organization and biogenesis	P
g22532.t1	GO:0000003	other biological processes	P
g21574.t1	GO:0000009	other molecular function	F
g15378.t1	GO:0000010	other molecular function	F
g5617.t1	GO:0000010	other molecular function	F


In [29]:
#join Blastquery-GOslim.tab with https://raw.githubusercontent.com/grace-ac/project-pycno-sizeclass-2022/main/analyses/08-deseq2/DEGlist_control_v_armdrop_7x7.tab in R
#Save Blastquery-GOslim.tab to directory

In [30]:
#check contents of directory
!ls

2015.Phel.BLASTx
Blastquery-GOslim.tab
DEGlist_7controlV7armdrop_blastGOslim.tab
GO-GOslim.sorted
Phel.genelist.BLASTx
_blast-GO-unfolded.sorted
_blast-GO-unfolded.tab
_blast-annot.tab
_blast-sep.tab
_blast-sort.tab
_intermediate.file
blastout
uniprot-SP-GO.sorted


In [32]:
!head -2 DEGlist_7controlV7armdrop_blastGOslim.tab

geneID	baseMean	log2FoldChange	lfcSE	stat	pvalue	padj	GOID	Biological_Process	P
g10002.t1	123.638504919869	0.728530719335967	0.284790842860068	2.55812550719522	0.0105238102589992	0.0335794643644705	NA	NA	NA


In [34]:
#count unique P rows (P = biological process)
!cat DEGlist_7controlV7armdrop_blastGOslim.tab | grep "	P"  | awk -F $'\t' '{print $10}' | sort | uniq -c

21657 P


In [36]:
#there are 21657 unique biological processes in the DEGlist... however the original DEGlist is only 7117 rows. 
#Reason is becase there are multiple GOslim terms per Phel gene

In [37]:
!cat DEGlist_7controlV7armdrop_blastGOslim.tab | grep "	P" | awk -F $'\t' '{print $9}' | sort | uniq -c | sort -r

3294 other biological processes
2849 developmental processes
2378 other metabolic processes
2288 cell organization and biogenesis
2051 protein metabolism
1832 transport
1642 RNA metabolism
1616 signal transduction
1366 stress response
 876 cell cycle and proliferation
 444 death
 420 DNA metabolism
 314 cell adhesion
 286 cell-cell signaling
   1 Biological_Process


In [38]:
!ls

2015.Phel.BLASTx
Blastquery-GOslim.tab
DEGlist_7controlV7armdrop_blastGOslim.tab
GO-GOslim.sorted
Phel.genelist.BLASTx
_blast-GO-unfolded.sorted
_blast-GO-unfolded.tab
_blast-annot.tab
_blast-sep.tab
_blast-sort.tab
_intermediate.file
blastout
uniprot-SP-GO.sorted
